In [10]:
import cv2
import os
import numpy as np
import time

In [11]:
THRESHOLD=0.3
CONFIDENCE=0.5

In [12]:
#The YOLOv3 was trained on the COCO dataset. Loading the classlabels from COCO
classLabels_file = '../Files/yolo-coco/coco.names'
labels = open(classLabels_file).read().strip().split('\n')
labels[:5]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane']

In [13]:
#Initializing Colors for representing different classes
COLORS = np.random.randint(0, 255, size=(len(labels), 3),dtype="uint8")
COLORS[:5]

array([[ 13, 172,  32],
       [ 83,  38, 174],
       [184,  64, 247],
       [253,   7,  60],
       [103, 180,  34]], dtype=uint8)

In [14]:
#Loading the YOLOv3 model. Since the original YOLO model was written in the Darknet framework, 
#we need to convert it to an OpenCV model.

weightsPath = '../Files/yolo-coco/yolov3.weights'
configPath = '../Files/yolo-coco/yolov3.cfg'

net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [6]:
#Loading the image
img = cv2.imread('../Files/images/soccer.jpg')
(H, W) = img.shape[:2]

In [15]:
# We need to get the layers with Unconnected Outputs.
ln = net.getLayerNames()
print(ln[:5])
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

['conv_0', 'bn_0', 'relu_1', 'conv_1', 'bn_1']
['yolo_82', 'yolo_94', 'yolo_106']


In [8]:
# construct a blob from the input image and then perform a forward. 
# A blob performs basic preprocessing on an image like mean subtraction.
# The standard input size for YOLOv3 is 416x416. Swapping the channels to suit the OpenCV format.
# pass of the YOLO object detector, giving us our bounding boxes and associated probabilities

blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
    swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
layerOutputs = net.forward(ln)
end = time.time()

print("Time Taken {:.6f}".format(end-start))

Time Taken 1.004279


In [9]:
# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively
boxes = []
confidences = []
classIDs = []

In [10]:
# For visualizing the results, we loop over each output
for output in layerOutputs:
    #Looping over each detection
    for detection in output:
        # extract the class ID and confidence (i.e., probability) of
        # the current object detection
        scores = detection[5:]
        #print(scores)
        
        classID = np.argmax(scores)
        confidence = scores[classID]
        
        # filter out weak predictions by ensuring the detected
        # probability is greater than the minimum probability
        if confidence > CONFIDENCE:
            #YOLO return the (x,y) of the center of the bounding box along with its width and height
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")
            
            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            
            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            

In [11]:
#Apply Non-Maxima Suppression to limit the number of boxes
idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE,THRESHOLD)

In [ ]:
# ensure at least one detection exists
if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
        # extract the bounding box coordinates
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])

        # draw a bounding box rectangle and label on the image
        color = [int(c) for c in COLORS[classIDs[i]]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(labels[classIDs[i]], confidences[i])
        cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
            0.5, color, 2)

In [ ]:
# show the output image
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# YOLOv3 on Videos

In [8]:
cap = cv2.VideoCapture('../Files/airport.mp4')
writer = None
(W, H) = (None, None)

#Counting the number of Frames
prop=cv2.CAP_PROP_FRAME_COUNT
total = int(cap.get(prop))
total

749

In [18]:
while True:
    ret, frame = cap.read()
    if(not ret):
        break
    
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    
    #Performing the same operations as on a single image
    # 1. Preprocessing 
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
    swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    
    boxes = []
    confidences = []
    classIDs = []
    
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            
            if confidence > CONFIDENCE:
            
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE,
        THRESHOLD)
        
        if len(idxs) > 0:
        # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])

                # draw a bounding box rectangle and label on the frame
                color = [int(c) for c in COLORS[classIDs[i]]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                text = "{}: {:.4f}".format(labels[classIDs[i]],
                    confidences[i])
                cv2.putText(frame, text, (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                cv2.imshow('Object Detection',frame)
                if cv2.waitKey(3)&0xFF == 27:
                    break

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()        